In [10]:
import pandas as pd
import numpy as np
from pymoo.optimize import minimize
import pickle

from pymoo.util.termination.f_tol import MultiObjectiveSpaceToleranceTermination
from pymoo.visualization.scatter import Scatter
from notebooks.optimization_problems.constraints import Requirements
from pyreport import PlotUtil
import matplotlib.pyplot as plt

%load_ext autoreload

%autoreload 2

plt.close()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
N_passes = 20

#res = pickle.load(open('pointing_res_nsga2.pkl' % (setting['label']), 'rb'))
#res = pickle.load(open('pointing_res_nsga3.pkl' % (setting['label']), 'rb'))
problem, setting, res = pickle.load(open('pointing_res_rnsga2.pkl', 'rb'))

FileNotFoundError: [Errno 2] No such file or directory: 'pointing_res_rnsga2.pkl'

In [ ]:
x_pass = res.X[:, problem.x_indices['pass']].astype('bool')
x_Ptx_dBm = res.X[:, problem.x_indices['power']].astype('float64')
x_Ptx_dBm[~x_pass] = np.NaN
x_Ptx_dBm = np.nanmax(x_Ptx_dBm, axis=1)
#x_modcod = np.squeeze(res.X[:, problem.x_indices['modcod']].astype('int64'))

f_pointing = res.F[:, 2]
f_energy = res.F[:,1] / 1e3 # Kilo Joule
f_throughput = ((res.F[:,0] * -1) / 1e9)   # Gigabit

In [ ]:
points = []
F_norm = res.F[:,1:3] / np.max(res.F[:,1:3], axis=0)

# Max throughput point
ind = np.argmax(f_throughput)
p = np.array([f_energy[ind], f_pointing[ind], f_throughput[ind]])
points.append({'ind': ind, 'p': p, 'color': 'r', 'marker': '.', 'size': 10, 'label': 'Max T\n(%d Gbit, %d kJ, %d s)' % (f_throughput[ind], f_energy[ind], f_pointing[ind])})

# Max E, minimum P point
ref = np.array([1, 0])
ind = np.argmin(np.linalg.norm(F_norm - ref, axis = 1))
p = np.array([f_energy[ind], f_pointing[ind], f_throughput[ind]])
points.append({'ind': ind, 'p': p, 'color': 'g', 'marker': '.', 'size': 10, 'label': 'Max E Min P\n(%d Gbit, %d kJ, %d s)' % (f_throughput[ind], f_energy[ind], f_pointing[ind])})

# # Max E, minimum P point
# ref = np.array([0.5, 0.5])
# ind = np.argmin(np.linalg.norm(F_norm - ref, axis = 1))
# p = np.array([f_energy[ind], f_pointing[ind], f_throughput[ind]])
# points.append({'ind': ind, 'p': p, 'color': 'b', 'label': 'Mid \n(%d Gbit, %d kJ, %d s)' % (f_throughput[ind], f_energy[ind], f_pointing[ind])})

# Reference points
ref_points = setting['algorithm'].survival.ref_points / np.array([-1e9, 1e3, 1])
points.append({'ind': ind, 'p': ref_points[0,:], 'color': 'g', 'marker': 'x', 'size': 10, 'label': 'Target 1\n(%d Gbit, %d kJ, %d s)' % (ref_points[0,0], ref_points[0,1], ref_points[0,2])})
points.append({'ind': ind, 'p': ref_points[1,:], 'color': 'g', 'marker': 'x', 'size': 10, 'label': 'Target 2\n(%d Gbit, %d kJ, %d s)' % (ref_points[1,0], ref_points[1,1], ref_points[1,2])})

In [ ]:
T_lim = (0, 1850)
E_lim = (0, 120)
P_lim = (0, 2000)

In [ ]:
%matplotlib
plt.close()

fig = plt.figure(figsize=(3.2, 2.4))
ax = fig.add_subplot(111, projection='3d', proj_type = 'ortho')

ax.scatter(f_energy, f_pointing, f_throughput, color='#1f77b4', marker='.', s=1)
for point in points:
    ax.scatter(point['p'][0], point['p'][1], point['p'][2],
               color=point['color'], s=point['size'], marker=point['marker'], label=point['label'])

ax.set_xlabel("Energy used [kJ]")
ax.set_ylabel("Pointing [s]")
ax.set_zlabel("Throughput [Gbit]")

ax.set_xlim(E_lim)
ax.set_ylim(P_lim)
ax.set_zlim(T_lim)

#ax.invert_xaxis()
#ax.invert_yaxis()

PlotUtil.apply_report_formatting_3D()
plt.tight_layout(pad=1.5, rect=(-0.1, -0.05, 1.15, 1.15))
ax.view_init(15, -140)

plt.ion()
#plt.show()

fig.savefig('D:/git/thesis_report_ae/figures/pointing_pareto_3d_%s.pdf' % setting['label'])

# Plot a legend
fig, ax = plt.subplots(figsize=(3.2, 2.4))

for point in points:
    ax.scatter(np.nan, np.nan, color=point['color'], s=10, label=point['label'])

ax.set_axis_off()
PlotUtil.apply_report_formatting()
ax.legend(fontsize=8)
fig.savefig('D:/git/thesis_report_ae/figures/pointing_pareto_3d_%s_leg.pdf' % setting['label'])

In [ ]:
fig = plt.figure(figsize=(3.2*3, 2.4))
axs = fig.subplots(1,3)

ax = axs[0]
ax.grid(True)
ax.scatter(f_energy, f_throughput, marker='.', s=1)
for point in points:
    ax.scatter(point['p'][0], point['p'][2],
               color=point['color'], s=point['size'], marker=point['marker'], label=point['label'])


# for d in np.arange(len(f_throughput)):
#     ax.text(f_energy[d], f_throughput[d], '%d' % d)

ax.set_xlabel("Energy used [kJ]")
ax.set_ylabel("Throughput [Gbit]")
ax.set_xlim(E_lim)
ax.set_ylim(T_lim)

ax = axs[1]
ax.grid(True)
ax.scatter(f_pointing, f_throughput, marker='.', s=1)
for point in points:
    ax.scatter(point['p'][1], point['p'][2],
               color=point['color'], s=point['size'], marker=point['marker'], label=point['label'])

ax.set_xlabel("Pointing [s]")
ax.set_ylabel("Throughput [Gbit]")
ax.set_xlim(P_lim)
ax.set_ylim(T_lim)

ax = axs[2]
ax.grid(True)
ax.scatter(f_energy, f_pointing, marker='.', s=1)
for point in points:
    ax.scatter(point['p'][0], point['p'][1],
               color=point['color'], s=point['size'], marker=point['marker'], label=point['label'])

# ax.scatter(p_maxTminP[0], p_maxTminP[1], color='b', s=10)
# ax.scatter(p_minEminP[0], p_minEminP[1], color='y', s=10)
ax.set_xlabel("Energy used [kJ]")
ax.set_ylabel("Pointing [s]")
ax.set_xlim(E_lim)
ax.set_ylim(P_lim)

plt.tight_layout()

fig.savefig('D:/git/thesis_report_ae/figures/pointing_pareto_tri_%s.pdf' % setting['label'])

In [ ]:
fig = plt.figure(figsize=(3.2, 2.4))
ax = fig.add_subplot(111, projection='3d', proj_type = 'ortho')

ref_points_plt = np.abs(ref_points / np.max(ref_points))

ax.scatter(0, 1, 1, color='g', marker='x', s=20)
ax.scatter(1, 0, 1, color='r', marker='x', s=20)

ax.set_xlabel("Energy used [kJ]")
ax.set_zlabel("Throughput [Gbit]")
ax.set_ylabel("Pointing [s]")

ax.set_xlim((0,1))
ax.set_ylim((0,1))
ax.set_zlim((0,1))

PlotUtil.apply_report_formatting_3D()
plt.tight_layout(pad=1.5, rect=(-0.1, -0.05, 1.15, 1.15))
ax.view_init(20, -135)

plt.ion()
#plt.show()

fig.savefig('D:/git/thesis_report_ae/figures/pointing_reference_points.pdf')

In [ ]:
print('ax.azim {}'.format(ax.azim))
print('ax.elev {}'.format(ax.elev))